In [1]:
# =================== MONUK FINAL WEATHER APP (WITH BUTTON ICONS) ====================
import tkinter as tk
from tkcalendar import Calendar
from datetime import datetime, timedelta
import speech_recognition as sr
import webbrowser
import customtkinter as ctk
from PIL import Image, ImageTk
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

root = ctk.CTk()
root.title("Cloudy Sky")
root.geometry("1200x750")


bg_img = Image.open("Background _image.jpg.jpeg").resize((2200, 2200))
bg_photo = ImageTk.PhotoImage(bg_img)
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

def update_time():
    time_label.configure(text=datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
    root.after(1000, update_time)

def show_calendar():
    calendar_frame.place(relx=0.75, rely=0.4)

def confirm_date():
    calendar_frame.place_forget()
    selected_date_label.configure(text=f"Selected Date: {calendar.get_date()}")

def get_voice_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        try:
            city_entry.delete(0, tk.END)
            city_entry.insert(0, "Listening...")
            audio = recognizer.listen(source, timeout=5)
            text = recognizer.recognize_google(audio)
            city_entry.delete(0, tk.END)
            city_entry.insert(0, text)
        except:
            city_entry.delete(0, tk.END)
            city_entry.insert(0, "Try Again")

def update_background(weather_main, weather_icon):
    print(f"Weather condition: {weather_main}, Icon: {weather_icon}")

def on_search():
    selected_city = city_entry.get()
    selected_date = calendar.get_date()
    selected_date_label.configure(text=f"Selected Date: {selected_date}")
    if not selected_city.strip():
        return
    api_key = "2891c36c237a9a499b8b995f208fdde2"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={selected_city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        data = response.json()
        if data["cod"] == 200:
            weather_main = data["weather"][0]["main"]
            weather_icon = data["weather"][0]["icon"]
            update_background(weather_main, weather_icon)
        else:
            city_entry.delete(0, tk.END)
            city_entry.insert(0, "City not found")
    except Exception as e:
        print("Error:", e)
        city_entry.delete(0, tk.END)
        city_entry.insert(0, "Network Error")

def show_main():
    for frame in [weather_container, map_container, news_container]:
        frame.place_forget()
    main_frame.place(relx=0.5, rely=0.5, anchor="center")

def show_weather():
    show_main()
    weather_container.place(relx=0.5, rely=0.5, anchor="center")
    for widget in weather_scroll_frame.winfo_children():
        if widget != weather_back:
            widget.destroy()
    city = city_entry.get()
    if not city.strip():
        ctk.CTkLabel(weather_scroll_frame, text="Enter city first!", text_color="red").pack()
        return
    api_key = "2891c36c237a9a499b8b995f208fdde2"
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        data = response.json()
        if data["cod"] == "200":
            weather_back.pack(anchor="w", padx=10, pady=10)
            city_name = data["city"]["name"]
            country = data["city"]["country"]
            ctk.CTkLabel(weather_scroll_frame, text=f"{city_name}, {country}", font=("Arial", 20, "bold"), text_color="blue").pack(pady=5)
            desc = data["list"][0]["weather"][0]["description"].title()
            temp = data["list"][0]["main"]["temp"]
            ctk.CTkLabel(weather_scroll_frame, text=f"\U0001F325 {desc} | {temp:.2f}°C", font=("Arial", 14)).pack()
            times = [item["dt_txt"][11:16] for item in data["list"][:12]]
            temps = [item["main"]["temp"] for item in data["list"][:12]]
            fig, ax = plt.subplots(figsize=(8, 3.5), dpi=100)
            ax.plot(times, temps, color="orange", marker="o")
            ax.set_title("Hourly Temperature")
            ax.set_ylabel("°C")
            ax.tick_params(axis='x', rotation=45)
            fig.tight_layout()
            chart = FigureCanvasTkAgg(fig, master=weather_scroll_frame)
            chart.draw()
            chart.get_tk_widget().pack(pady=10)
            forecast_frame = tk.Frame(weather_scroll_frame, bg="#f2ffff")
            forecast_frame.pack(pady=10)


            for i in range(24):
                item = data["list"][i]
                time_24 = item["dt_txt"][11:16]
                hour = datetime.strptime(time_24, "%H:%M").strftime("%I:%M %p")
                temp = item["main"]["temp"]
                icon = item["weather"][0]["icon"]
                img_url = f"http://openweathermap.org/img/wn/{icon}.png"
                icon_img = Image.open(BytesIO(requests.get(img_url).content)).resize((50, 50))
                icon_photo = ImageTk.PhotoImage(icon_img)
                card = tk.Frame(forecast_frame, bg="#dff6ff", bd=1, relief="solid")
                card.grid(row=i//6, column=i%6, padx=5, pady=5)
                tk.Label(card, text=hour, font=("Arial", 10), bg="#dff6ff").pack()
                tk.Label(card, image=icon_photo, bg="#dff6ff").pack()
                tk.Label(card, text=f"{temp:.1f}°C", font=("Arial", 10), bg="#dff6ff").pack()
                card.image = icon_photo
        else:
            ctk.CTkLabel(weather_scroll_frame, text="City not found!", text_color="red").pack()
    except Exception as e:
       ctk.CTkLabel(weather_scroll_frame, text=f"Error: {e}", text_color="red").pack()


# ==== MAP ====
def show_map():
    city = city_entry.get()
    if city.strip():
        webbrowser.open(f"https://www.google.com/maps/place/{city}")


# ==== NEWS ====
def show_news():
    news_container.place(relx=0.5, rely=0.6, anchor="center")

    city = city_entry.get().strip()
    if not city:
        ctk.CTkLabel(news_scroll_frame, text="Please enter city", text_color="red").pack()
        return

    date_text = selected_date_label.cget("text")
    if "Selected Date:" in date_text:
        date_str = date_text.replace("Selected Date:", "").strip()
        try:
            date_obj = datetime.strptime(date_str, "%d-%m-%Y")
            from_date = (date_obj - timedelta(days=1)).strftime("%Y-%m-%d")
            to_date = (date_obj + timedelta(days=1)).strftime("%Y-%m-%d")
        except ValueError:
            ctk.CTkLabel(news_scroll_frame, text="Invalid date format", text_color="red").pack()
            return
    else:
        ctk.CTkLabel(news_scroll_frame, text="Please select a date", text_color="red").pack()
        return

    # Clear previous news
    for widget in news_scroll_frame.winfo_children():
        if widget != news_back:
            widget.destroy()

    news_back.pack(anchor="w", padx=10, pady=10)

    api_key = "fddf7cda28b6424cb8a4ee36162c019d"

    def fetch_news(query, label_text):
        url = (
            f"https://newsapi.org/v2/everything?"
            f"q={query}&from={from_date}&to={to_date}&sortBy=publishedAt&"
            f"apiKey={api_key}&language=en"
        )
        try:
            response = requests.get(url)
            data = response.json()

            ctk.CTkLabel(news_scroll_frame, text=label_text, font=("Arial", 18, "bold"), text_color="#333").pack(pady=(10, 5))

            if data["status"] == "ok":
                articles = data["articles"][:5]
                if not articles:
                    ctk.CTkLabel(news_scroll_frame, text="No news found.", text_color="orange").pack(pady=5)
                for article in articles:
                    title = article["title"]
                    published_at = article.get("publishedAt", "")[:10]
                    link = article["url"]
                    news_text = f"📰 {title} ({published_at})"
                    label = ctk.CTkLabel(news_scroll_frame, text=news_text, font=("Arial", 14), text_color="#0000ee", cursor="hand2", anchor="w", justify="left")
                    label.pack(fill="x", padx=10, pady=2)
                    label.bind("<Button-1>", lambda e, url=link: webbrowser.open(url))
            else:
                ctk.CTkLabel(news_scroll_frame, text="Error fetching news.", text_color="red").pack(pady=5)
        except Exception as e:
            ctk.CTkLabel(news_scroll_frame, text=f"Error: {e}", text_color="red").pack(pady=5)

    # Fetch city-related news
    fetch_news(city, f"🧭 {city.title()} Related News")

    # Fetch weather-related news
    fetch_news(f"{city} weather", f"🌤 Weather News for {city.title()}")


# ========== GUI LAYOUT ============
main_frame = ctk.CTkFrame(root,fg_color="transparent",bg_color="transparent")
main_frame.place(relx=0.5, rely=0.5, anchor="center")

title_frame = tk.Frame(root, bg="#e3f2fd")
title_frame.pack(pady=10)
logo_image = Image.open("sky_logo.png.png").resize((60, 60))
logo_photo = ImageTk.PhotoImage(logo_image)
logo_label = tk.Label(title_frame, image=logo_photo)

logo_label.pack(side="left", padx=5)
logo_label.image = logo_photo
ctk.CTkLabel(title_frame, text="Weather App", font=("Arial", 42, "bold"), text_color="#004d66").pack(side="left")

time_label = ctk.CTkLabel(main_frame, font=("Arial", 22), text_color="#003366")
time_label.pack()
update_time()

input_outer = ctk.CTkFrame(main_frame, fg_color="transparent",bg_color="transparent", corner_radius=20)
input_outer.pack(pady=15, padx=20)
input_frame = ctk.CTkFrame(input_outer, fg_color="transparent",bg_color="transparent")
input_frame.pack(padx=10, pady=10)

city_entry = ctk.CTkEntry(input_frame, font=("Arial", 14), width=350, height=40, corner_radius=20, placeholder_text="Enter City")
city_entry.grid(row=0, column=0, padx=5)
ctk.CTkButton(input_frame, text="🎤", width=120, height=60, font=("Arial", 20), fg_color="#ff6666", text_color="white", command=get_voice_input, corner_radius=20).grid(row=0, column=1, padx=5)
ctk.CTkButton(input_frame, text="Search", width=250, height=60, font=("Arial", 22), fg_color="#3399ff", text_color="white", command=on_search, corner_radius=20).grid(row=0, column=2, padx=5)
ctk.CTkButton(input_frame, text="📅", width=200, height=60, font=("Arial", 22), fg_color="#6495ed", text_color="white", command=show_calendar, corner_radius=20).grid(row=0, column=3, padx=5)

selected_date_label = ctk.CTkLabel(input_outer, text="Selected Date:", font=("Arial", 18), text_color="#003366")
selected_date_label.pack(pady=5)

calendar_frame = tk.Frame(root, bg="white", bd=2, relief="ridge")
calendar = Calendar(calendar_frame, selectmode='day', date_pattern='dd-mm-yyyy')
calendar.pack(padx=10, pady=10)
tk.Button(calendar_frame, text="OK", command=confirm_date).pack(pady=5)
calendar_frame.place_forget()

# === LOAD BUTTON ICONS ===
weather_icon = Image.open("weather icon.png.png").resize((80, 80))
map_icon = Image.open("location icon.png.png").resize((80, 80))
news_icon = Image.open("news icon.png.png").resize((80, 80))
weather_photo = ImageTk.PhotoImage(weather_icon)
map_photo = ImageTk.PhotoImage(map_icon)
news_photo = ImageTk.PhotoImage(news_icon)

card_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
card_frame.pack(pady=20)

weather_btn = ctk.CTkButton(card_frame, text="Weather", image=weather_photo, compound="left", font=("Arial", 22, "bold"), width=550, height=80, fg_color="#3399ff", text_color="white", corner_radius=40, command=show_weather)
weather_btn.pack(pady=12)
weather_btn.image = weather_photo

map_btn = ctk.CTkButton(card_frame, text="Map", image=map_photo, compound="left", font=("Arial", 22, "bold"), width=550, height=80, fg_color="#66cc99", text_color="white", corner_radius=40, command=show_map)
map_btn.pack(pady=12)
map_btn.image = map_photo

news_btn = ctk.CTkButton(card_frame, text="News", image=news_photo, compound="left", font=("Arial", 22, "bold"), width=550, height=80, fg_color="#ff9999", text_color="white", corner_radius=40, command=show_news)
news_btn.pack(pady=12)
news_btn.image = news_photo

weather_container = ctk.CTkFrame(root, fg_color="#f2ffff", width=900, height=700)
weather_container.pack_propagate(False)

weather_scroll_frame = ctk.CTkScrollableFrame(weather_container, width=1080,height=630, fg_color="#f2ffff")
weather_scroll_frame.pack(fill="both", expand=True)

weather_back = ctk.CTkButton(weather_scroll_frame, text="🔙 Back", font=("Arial", 12), command=show_main)

map_container = ctk.CTkFrame(root, fg_color="#f2fff5", width=600, height=600)
map_container.pack_propagate(False)
map_scroll_frame = ctk.CTkScrollableFrame(map_container, width=580, height=580, fg_color="#f2fff5")
map_scroll_frame.pack(fill="both", expand=True)
map_back = ctk.CTkButton(map_scroll_frame, text="🔙 Back", font=("Arial", 12), command=show_main)

news_container = ctk.CTkFrame(root, fg_color="#fff5f5", width=600, height=600)
news_container.pack_propagate(False)
news_scroll_frame = ctk.CTkScrollableFrame(news_container, width=580, height=580, fg_color="#fff5f5")
news_scroll_frame.pack(fill="both", expand=True)
news_back = ctk.CTkButton(news_scroll_frame, text="🔙 Back", font=("Arial", 12), command=show_main)

root.mainloop()

c:\Users\Lenovo\anaconda3\Lib\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkButton Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")


In [2]:
################ Teju's Weather App – News1 ################   add button  logos #############

import tkinter as tk
from tkcalendar import Calendar
from datetime import datetime
import speech_recognition as sr
import webbrowser
import customtkinter as ctk
from PIL import Image, ImageTk
import requests
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from io import BytesIO

ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

root = ctk.CTk()
root.title("Cloudy Sky")
root.geometry("600x750")
root.configure(fg_color="#e3f2fd")

# ==== Load Initial Background ====
bg_img = Image.open("default.jpg").resize((600, 750))
bg_photo = ImageTk.PhotoImage(bg_img)
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

# ==== Time ====
def update_time():
    time_label.configure(text=datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
    root.after(1000, update_time)

# ==== Calendar ====
def show_calendar():
    calendar_frame.place(relx=0.75, rely=0.4)

def confirm_date():
    calendar_frame.place_forget()
    selected_date_label.configure(text=f"Selected Date: {calendar.get_date()}")

# ==== Voice Input ====
def get_voice_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        try:
            city_entry.delete(0, tk.END)
            city_entry.insert(0, "Listening...")
            audio = recognizer.listen(source, timeout=5)
            text = recognizer.recognize_google(audio)
            city_entry.delete(0, tk.END)
            city_entry.insert(0, text)
        except:
            city_entry.delete(0, tk.END)
            city_entry.insert(0, "Try Again")

# ==== Background Change ====
def update_background(weather_main, weather_icon):
    is_day = 'd' in weather_icon.lower()
    if weather_main.lower() == "clear":
        bg_path = "sunny.jpg" if is_day else "clear_night.jpg"
    elif weather_main.lower() == "clouds":
        bg_path = "cloud.jpg"
    elif weather_main.lower() == "rain":
        bg_path = "rain.jpg"
    elif weather_main.lower() == "thunderstorm":
        bg_path = "thunderstorm.jpg"
    elif weather_main.lower() == "snow":
        bg_path = "snow.jpg"
    elif weather_main.lower() in ["mist", "fog"]:
        bg_path = "fog.jpg"
    else:
        bg_path = "default.jpg"
    try:
        new_bg = Image.open(bg_path).resize((600, 750))
        new_bg_photo = ImageTk.PhotoImage(new_bg)
        bg_label.configure(image=new_bg_photo)
        bg_label.image = new_bg_photo
    except Exception as e:
        print("Image loading error:", e)

# ==== Search Only Updates Background ====
def on_search():
    selected_city = city_entry.get()
    selected_date = calendar.get_date()
    selected_date_label.configure(text=f"Selected Date: {selected_date}")
    if not selected_city.strip():
        return

    api_key = "2891c36c237a9a499b8b995f208fdde2"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={selected_city}&appid={api_key}&units=metric"

    try:
        response = requests.get(url)
        data = response.json()
        if data["cod"] == 200:
            weather_main = data["weather"][0]["main"]
            weather_icon = data["weather"][0]["icon"]
            update_background(weather_main, weather_icon)
        else:
            city_entry.delete(0, tk.END)
            city_entry.insert(0, "City not found")
    except Exception as e:
        print("Error:", e)
        city_entry.delete(0, tk.END)
        city_entry.insert(0, "Network Error")

# ==== Main Hide Logic ====
def show_main():
    for frame in [weather_container, news_container]:
        frame.place_forget()

# ==== WEATHER Full Forecast ====
def show_weather():
    show_main()
    weather_container.place(relx=0.5, rely=0.5, anchor="center")

    for widget in weather_scroll_frame.winfo_children():
        if widget != weather_back:
            widget.destroy()

    city = city_entry.get()
    if not city.strip():
        ctk.CTkLabel(weather_scroll_frame, text="Enter city first!", text_color="red").pack()
        return

    api_key = "2891c36c237a9a499b8b995f208fdde2"
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"

    try:
        response = requests.get(url)
        data = response.json()

        if data["cod"] == "200":
            weather_back.pack(anchor="w", padx=10, pady=10)

            city_name = data["city"]["name"]
            country = data["city"]["country"]
            ctk.CTkLabel(weather_scroll_frame, text=f"{city_name}, {country}", font=("Arial", 20, "bold"), text_color="blue").pack(pady=5)

            desc = data["list"][0]["weather"][0]["description"].title()
            temp = data["list"][0]["main"]["temp"]
            ctk.CTkLabel(weather_scroll_frame, text=f"\U0001F325 {desc} | {temp:.2f}°C", font=("Arial", 14)).pack()

            # Line Chart
            times = [item["dt_txt"][11:16] for item in data["list"][:12]]
            temps = [item["main"]["temp"] for item in data["list"][:12]]
            fig, ax = plt.subplots(figsize=(5, 2.5), dpi=100)
            ax.plot(times, temps, color="orange", marker="o")
            ax.set_title("Hourly Temperature")
            ax.set_ylabel("°C")
            ax.tick_params(axis='x', rotation=45)
            fig.tight_layout()

            chart = FigureCanvasTkAgg(fig, master=weather_scroll_frame)
            chart.draw()
            chart.get_tk_widget().pack(pady=10)

            # Forecast Cards
            card_frame = tk.Frame(weather_scroll_frame, bg="#e3f2fd")
            card_frame.pack()

            for i in range(12):
                item = data["list"][i]
                hour = item["dt_txt"][11:16]
                temp = item["main"]["temp"]
                icon = item["weather"][0]["icon"]
                img_url = f"http://openweathermap.org/img/wn/{icon}.png"
                icon_img = Image.open(BytesIO(requests.get(img_url).content)).resize((30, 30))
                icon_photo = ImageTk.PhotoImage(icon_img)

                card = tk.Frame(card_frame, bg="#dff6ff", bd=1, relief="solid")
                card.pack(side="left", padx=5, pady=5)
                tk.Label(card, text=hour, font=("Arial", 10), bg="#dff6ff").pack()
                tk.Label(card, image=icon_photo, bg="#dff6ff").pack()
                tk.Label(card, text=f"{temp:.1f}°C", font=("Arial", 10), bg="#dff6ff").pack()
                card.image = icon_photo
        else:
            ctk.CTkLabel(weather_scroll_frame, text="City not found!", text_color="red").pack()
    except Exception as e:
        ctk.CTkLabel(weather_scroll_frame, text=f"Error: {e}", text_color="red").pack()

# ==== MAP ====
def show_map():
    city = city_entry.get()
    if city.strip():
        webbrowser.open(f"https://www.google.com/maps/place/{city}")

# ==== NEWS ====
def show_news():
    show_main()
    news_container.place(relx=0.5, rely=0.5, anchor="center")
    city = city_entry.get()

    for widget in news_scroll_frame.winfo_children():
        if widget != news_back:
            widget.destroy()

    news_back.pack(anchor="w", padx=10, pady=10)
    ctk.CTkLabel(news_scroll_frame, text=f"News for {city}", font=("Arial", 20, "bold"), text_color="#004d66").pack(pady=10)

    if not city.strip():
        ctk.CTkLabel(news_scroll_frame, text="Enter a city first", text_color="red").pack()
        return

    api_key = "fddf7cda28b6424cb8a4ee36162c019d"
    url = f"https://newsapi.org/v2/everything?q={city}&sortBy=publishedAt&apiKey={api_key}&language=en"

    try:
        response = requests.get(url)
        data = response.json()

        if data["status"] == "ok":
            articles = data["articles"][:10]
            if not articles:
                ctk.CTkLabel(news_scroll_frame, text="No news found for this city.", text_color="orange").pack()
            for article in articles:
                title = article["title"]
                link = article["url"]
                headline = f"🗞️ {title}"
                label = ctk.CTkLabel(news_scroll_frame, text=headline, font=("Arial", 14), text_color="#0000ee", cursor="hand2", anchor="w", justify="left")
                label.pack(fill="x", padx=10, pady=2)
                label.bind("<Button-1>", lambda e, url=link: webbrowser.open(url))
        else:
            ctk.CTkLabel(news_scroll_frame, text="Error fetching news.", text_color="red").pack()
    except Exception as e:
        ctk.CTkLabel(news_scroll_frame, text=f"Error: {e}", text_color="red").pack()

# ==== UI Layout ====
title_frame = tk.Frame(root, bg="#e3f2fd")
title_frame.pack(pady=10)

logo_image = Image.open("sky_logo.png").resize((40, 40))
logo_photo = ImageTk.PhotoImage(logo_image)
tk.Label(title_frame, image=logo_photo, bg="#e3f2fd").pack(side="left", padx=5)
ctk.CTkLabel(title_frame, text="Weather App", font=("Arial", 26, "bold"), text_color="#004d66").pack(side="left")

time_label = ctk.CTkLabel(root, font=("Arial", 12), text_color="#003366")
time_label.pack()
update_time()

input_outer = ctk.CTkFrame(root, fg_color="white", corner_radius=20)
input_outer.pack(pady=15, padx=20)

input_frame = ctk.CTkFrame(input_outer, fg_color="white")
input_frame.pack(padx=10, pady=10)

city_entry = ctk.CTkEntry(input_frame, font=("Arial", 14), width=200, height=40, corner_radius=20, placeholder_text="Enter City")
city_entry.grid(row=0, column=0, padx=5)

ctk.CTkButton(input_frame, text="🎤", width=40, height=40, font=("Arial", 12), fg_color="#ff6666", text_color="white", command=get_voice_input, corner_radius=20).grid(row=0, column=1, padx=5)
ctk.CTkButton(input_frame, text="Search", width=80, height=40, font=("Arial", 12), fg_color="#3399ff", text_color="white", command=on_search, corner_radius=20).grid(row=0, column=2, padx=5)
ctk.CTkButton(input_frame, text="📅", width=40, height=40, font=("Arial", 12), fg_color="#6495ed", text_color="white", command=show_calendar, corner_radius=20).grid(row=0, column=3, padx=5)

selected_date_label = ctk.CTkLabel(input_outer, text="Selected Date:", font=("Arial", 14), text_color="#003366")
selected_date_label.pack(pady=5)

calendar_frame = tk.Frame(root, bg="white", bd=2, relief="ridge")
calendar = Calendar(calendar_frame, selectmode='day', date_pattern='dd-mm-yyyy')
calendar.pack(padx=10, pady=10)
tk.Button(calendar_frame, text="OK", bg="#dcdcdc", command=confirm_date).pack(pady=5)
calendar_frame.place_forget()

# ==== Load Button Icons ====
map_icon_image = Image.open("map_icon.png").resize((25, 25))
map_icon_photo = ImageTk.PhotoImage(map_icon_image)

weather_icon_image = Image.open("weather_man_logo.jpg").resize((25, 25))
weather_icon_photo = ImageTk.PhotoImage(weather_icon_image)

news_icon_image = Image.open("news_logo.jpg").resize((25, 25))
news_icon_photo = ImageTk.PhotoImage(news_icon_image)

# ==== Buttons ====
ctk.CTkButton(root, text="Weather", image=weather_icon_photo, compound="left", font=("Arial", 20, "bold"), width=350, height=80, fg_color="#3399ff", text_color="white", corner_radius=40, command=show_weather).pack(pady=12)
ctk.CTkButton(root, text="Map", image=map_icon_photo, compound="left", font=("Arial", 20, "bold"), width=350, height=80, fg_color="#66cc99", text_color="white", corner_radius=40, command=show_map).pack(pady=12)
ctk.CTkButton(root, text="News", image=news_icon_photo, compound="left", font=("Arial", 20, "bold"), width=350, height=80, fg_color="#ff9999", text_color="white", corner_radius=40, command=show_news).pack(pady=10)

# ==== Containers ====
weather_container = ctk.CTkFrame(root, fg_color="#f2ffff", width=600, height=600)
weather_container.pack_propagate(False)
weather_scroll_frame = ctk.CTkScrollableFrame(weather_container, width=580, height=580, fg_color="#f2ffff")
weather_scroll_frame.pack(fill="both", expand=True)
weather_back = ctk.CTkButton(weather_scroll_frame, text="🔙 Back", font=("Arial", 12), command=show_main)

news_container = ctk.CTkFrame(root, fg_color="#fff5f5", width=600, height=600)
news_container.pack_propagate(False)
news_scroll_frame = ctk.CTkScrollableFrame(news_container, width=580, height=580, fg_color="#fff5f5")
news_scroll_frame.pack(fill="both", expand=True)
news_back = ctk.CTkButton(news_scroll_frame, text="🔙 Back", font=("Arial", 12), command=show_main)

root.mainloop()

FileNotFoundError: [Errno 2] No such file or directory: 'default.jpg'

In [1]:
# =================== MONUK FINAL WEATHER APP (WITH BUTTON ICONS) ====================
import tkinter as tk
from tkcalendar import Calendar
from datetime import datetime, timedelta
import speech_recognition as sr
import webbrowser
import customtkinter as ctk
from PIL import Image, ImageTk
import requests
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import mysql.connector
from mysql.connector import Error

import mysql.connector
from mysql.connector import Error

# Step 1: Create Database
def create_database(db_name="weather_db"):
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Teju1234"
        )
        cursor = connection.cursor()
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        cursor.close()
        connection.close()
    except Error as e:
        print(f"Error creating database: {e}")

# Step 2: Connect to Database
def create_connection(db_name="weather_db"):
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="Teju1234",
            database=db_name
        )
        if connection.is_connected():
            return connection
    except Error as e:
        print(f"Error connecting to database: {e}")
        return None

# Step 3: Create Table
def create_table():
    connection = create_connection()
    if connection:
        cursor = connection.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS weather_history (
                id INT AUTO_INCREMENT PRIMARY KEY,
                city VARCHAR(100),
                temperature FLOAT,
                description VARCHAR(255),
                search_time DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        """)
        connection.commit()
        cursor.close()
        connection.close()


# ------------------ Run Setup Once ------------------
create_database()   # Creates database if not exists
create_table()      # Creates table if not exists


def insert_weather(city, temperature, description):
    connection = create_connection()
    if connection:
        cursor = connection.cursor()
        cursor.execute("""
            INSERT INTO weather_history (city, temperature, description)
            VALUES (%s, %s, %s)
        """, (city, temperature, description))
        connection.commit()
        cursor.close()
        connection.close()


ctk.set_appearance_mode("light")
ctk.set_default_color_theme("blue")

root = ctk.CTk()
root.title("Cloudy Sky")
root.geometry("1200x750")


bg_img = Image.open("Background _image.jpg.jpeg").resize((2200, 2200))
bg_photo = ImageTk.PhotoImage(bg_img)
bg_label = tk.Label(root, image=bg_photo)
bg_label.place(x=0, y=0, relwidth=1, relheight=1)

def update_time():
    time_label.configure(text=datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
    root.after(1000, update_time)

def show_calendar():
    calendar_frame.place(relx=0.75, rely=0.4)

def confirm_date():
    calendar_frame.place_forget()
    selected_date_label.configure(text=f"Selected Date: {calendar.get_date()}")

def get_voice_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        try:
            city_entry.delete(0, tk.END)
            city_entry.insert(0, "Listening...")
            audio = recognizer.listen(source, timeout=5)
            text = recognizer.recognize_google(audio)
            city_entry.delete(0, tk.END)
            city_entry.insert(0, text)
        except:
            city_entry.delete(0, tk.END)
            city_entry.insert(0, "Try Again")

def update_background(weather_main, weather_icon):
    print(f"Weather condition: {weather_main}, Icon: {weather_icon}")

def on_search():
    selected_city = city_entry.get()
    selected_date = calendar.get_date()
    selected_date_label.configure(text=f"Selected Date: {selected_date}")
    if not selected_city.strip():
        return
    api_key = "2891c36c237a9a499b8b995f208fdde2"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={selected_city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        data = response.json()
        if data["cod"] == 200:
            weather_main = data["weather"][0]["main"]
            weather_icon = data["weather"][0]["icon"]
            temperature = data["main"]["temp"]                         # <-- add this line
            description = data["weather"][0]["description"].title()
                       # ----- Step 6: Save to MySQL -----
            insert_weather(selected_city, temperature, description)  # <-- THIS IS STEP 6
            update_background(weather_main, weather_icon)
        else:
            city_entry.delete(0, tk.END)
            city_entry.insert(0, "City not found")
    except Exception as e:
        print("Error:", e)
        city_entry.delete(0, tk.END)
        city_entry.insert(0, "Network Error")

def show_main():
    for frame in [weather_container, map_container, news_container]:
        frame.place_forget()
    main_frame.place(relx=0.5, rely=0.5, anchor="center")

def show_weather():
    show_main()
    weather_container.place(relx=0.5, rely=0.5, anchor="center")
    for widget in weather_scroll_frame.winfo_children():
        if widget != weather_back:
            widget.destroy()
    city = city_entry.get()
    if not city.strip():
        ctk.CTkLabel(weather_scroll_frame, text="Enter city first!", text_color="red").pack()
        return
    api_key = "2891c36c237a9a499b8b995f208fdde2"
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    try:
        response = requests.get(url)
        data = response.json()
        if data["cod"] == "200":
            weather_back.pack(anchor="w", padx=10, pady=10)
            city_name = data["city"]["name"]
            country = data["city"]["country"]
            ctk.CTkLabel(weather_scroll_frame, text=f"{city_name}, {country}", font=("Arial", 20, "bold"), text_color="blue").pack(pady=5)
            desc = data["list"][0]["weather"][0]["description"].title()
            temp = data["list"][0]["main"]["temp"]
            ctk.CTkLabel(weather_scroll_frame, text=f"\U0001F325 {desc} | {temp:.2f}°C", font=("Arial", 14)).pack()
            times = [item["dt_txt"][11:16] for item in data["list"][:12]]
            temps = [item["main"]["temp"] for item in data["list"][:12]]
            fig, ax = plt.subplots(figsize=(8, 3.5), dpi=100)
            ax.plot(times, temps, color="orange", marker="o")
            ax.set_title("Hourly Temperature")
            ax.set_ylabel("°C")
            ax.tick_params(axis='x', rotation=45)
            fig.tight_layout()
            chart = FigureCanvasTkAgg(fig, master=weather_scroll_frame)
            chart.draw()
            chart.get_tk_widget().pack(pady=10)
            forecast_frame = tk.Frame(weather_scroll_frame, bg="#f2ffff")
            forecast_frame.pack(pady=10)


            for i in range(24):
                item = data["list"][i]
                time_24 = item["dt_txt"][11:16]
                hour = datetime.strptime(time_24, "%H:%M").strftime("%I:%M %p")
                temp = item["main"]["temp"]
                icon = item["weather"][0]["icon"]
                img_url = f"http://openweathermap.org/img/wn/{icon}.png"
                icon_img = Image.open(BytesIO(requests.get(img_url).content)).resize((50, 50))
                icon_photo = ImageTk.PhotoImage(icon_img)
                card = tk.Frame(forecast_frame, bg="#dff6ff", bd=1, relief="solid")
                card.grid(row=i//6, column=i%6, padx=5, pady=5)
                tk.Label(card, text=hour, font=("Arial", 10), bg="#dff6ff").pack()
                tk.Label(card, image=icon_photo, bg="#dff6ff").pack()
                tk.Label(card, text=f"{temp:.1f}°C", font=("Arial", 10), bg="#dff6ff").pack()
                card.image = icon_photo
        else:
            ctk.CTkLabel(weather_scroll_frame, text="City not found!", text_color="red").pack()
    except Exception as e:
       ctk.CTkLabel(weather_scroll_frame, text=f"Error: {e}", text_color="red").pack()


# ==== MAP ====
def show_map():
    city = city_entry.get()
    if city.strip():
        webbrowser.open(f"https://www.google.com/maps/place/{city}")


# ==== NEWS ====
def show_news():
    news_container.place(relx=0.5, rely=0.6, anchor="center")

    city = city_entry.get().strip()
    if not city:
        ctk.CTkLabel(news_scroll_frame, text="Please enter city", text_color="red").pack()
        return

    date_text = selected_date_label.cget("text")
    if "Selected Date:" in date_text:
        date_str = date_text.replace("Selected Date:", "").strip()
        try:
            date_obj = datetime.strptime(date_str, "%d-%m-%Y")
            from_date = (date_obj - timedelta(days=1)).strftime("%Y-%m-%d")
            to_date = (date_obj + timedelta(days=1)).strftime("%Y-%m-%d")
        except ValueError:
            ctk.CTkLabel(news_scroll_frame, text="Invalid date format", text_color="red").pack()
            return
    else:
        ctk.CTkLabel(news_scroll_frame, text="Please select a date", text_color="red").pack()
        return

    # Clear previous news
    for widget in news_scroll_frame.winfo_children():
        if widget != news_back:
            widget.destroy()

    news_back.pack(anchor="w", padx=10, pady=10)

    api_key = "fddf7cda28b6424cb8a4ee36162c019d"

    def fetch_news(query, label_text):
        url = (
            f"https://newsapi.org/v2/everything?"
            f"q={query}&from={from_date}&to={to_date}&sortBy=publishedAt&"
            f"apiKey={api_key}&language=en"
        )
        try:
            response = requests.get(url)
            data = response.json()

            ctk.CTkLabel(news_scroll_frame, text=label_text, font=("Arial", 18, "bold"), text_color="#333").pack(pady=(10, 5))

            if data["status"] == "ok":
                articles = data["articles"][:5]
                if not articles:
                    ctk.CTkLabel(news_scroll_frame, text="No news found.", text_color="orange").pack(pady=5)
                for article in articles:
                    title = article["title"]
                    published_at = article.get("publishedAt", "")[:10]
                    link = article["url"]
                    news_text = f"📰 {title} ({published_at})"
                    label = ctk.CTkLabel(news_scroll_frame, text=news_text, font=("Arial", 14), text_color="#0000ee", cursor="hand2", anchor="w", justify="left")
                    label.pack(fill="x", padx=10, pady=2)
                    label.bind("<Button-1>", lambda e, url=link: webbrowser.open(url))
            else:
                ctk.CTkLabel(news_scroll_frame, text="Error fetching news.", text_color="red").pack(pady=5)
        except Exception as e:
            ctk.CTkLabel(news_scroll_frame, text=f"Error: {e}", text_color="red").pack(pady=5)

    # Fetch city-related news
    fetch_news(city, f"🧭 {city.title()} Related News")

    # Fetch weather-related news
    fetch_news(f"{city} weather", f"🌤 Weather News for {city.title()}")


# ========== GUI LAYOUT ============
main_frame = ctk.CTkFrame(root,fg_color="transparent",bg_color="transparent")
main_frame.place(relx=0.5, rely=0.5, anchor="center")

title_frame = tk.Frame(root, bg="#e3f2fd")
title_frame.pack(pady=10)
logo_image = Image.open("sky_logo.png.png").resize((60, 60))
logo_photo = ImageTk.PhotoImage(logo_image)
logo_label = tk.Label(title_frame, image=logo_photo)

logo_label.pack(side="left", padx=5)
logo_label.image = logo_photo
ctk.CTkLabel(title_frame, text="Weather App", font=("Arial", 42, "bold"), text_color="#004d66").pack(side="left")

time_label = ctk.CTkLabel(main_frame, font=("Arial", 22), text_color="#003366")
time_label.pack()
update_time()

input_outer = ctk.CTkFrame(main_frame, fg_color="transparent",bg_color="transparent", corner_radius=20)
input_outer.pack(pady=15, padx=20)
input_frame = ctk.CTkFrame(input_outer, fg_color="transparent",bg_color="transparent")
input_frame.pack(padx=10, pady=10)

city_entry = ctk.CTkEntry(input_frame, font=("Arial", 14), width=350, height=40, corner_radius=20, placeholder_text="Enter City")
city_entry.grid(row=0, column=0, padx=5)
ctk.CTkButton(input_frame, text="🎤", width=120, height=60, font=("Arial", 20), fg_color="#ff6666", text_color="white", command=get_voice_input, corner_radius=20).grid(row=0, column=1, padx=5)
ctk.CTkButton(input_frame, text="Search", width=250, height=60, font=("Arial", 22), fg_color="#3399ff", text_color="white", command=on_search, corner_radius=20).grid(row=0, column=2, padx=5)
ctk.CTkButton(input_frame, text="📅", width=200, height=60, font=("Arial", 22), fg_color="#6495ed", text_color="white", command=show_calendar, corner_radius=20).grid(row=0, column=3, padx=5)

selected_date_label = ctk.CTkLabel(input_outer, text="Selected Date:", font=("Arial", 18), text_color="#003366")
selected_date_label.pack(pady=5)

calendar_frame = tk.Frame(root, bg="white", bd=2, relief="ridge")
calendar = Calendar(calendar_frame, selectmode='day', date_pattern='dd-mm-yyyy')
calendar.pack(padx=10, pady=10)
tk.Button(calendar_frame, text="OK", command=confirm_date).pack(pady=5)
calendar_frame.place_forget()

# === LOAD BUTTON ICONS ===
weather_icon = Image.open("weather icon.png.png").resize((80, 80))
map_icon = Image.open("location icon.png.png").resize((80, 80))
news_icon = Image.open("news icon.png.png").resize((80, 80))
weather_photo = ImageTk.PhotoImage(weather_icon)
map_photo = ImageTk.PhotoImage(map_icon)
news_photo = ImageTk.PhotoImage(news_icon)

card_frame = ctk.CTkFrame(main_frame, fg_color="transparent")
card_frame.pack(pady=20)

weather_btn = ctk.CTkButton(card_frame, text="Weather", image=weather_photo, compound="left", font=("Arial", 22, "bold"), width=550, height=80, fg_color="#3399ff", text_color="white", corner_radius=40, command=show_weather)
weather_btn.pack(pady=12)
weather_btn.image = weather_photo

map_btn = ctk.CTkButton(card_frame, text="Map", image=map_photo, compound="left", font=("Arial", 22, "bold"), width=550, height=80, fg_color="#66cc99", text_color="white", corner_radius=40, command=show_map)
map_btn.pack(pady=12)
map_btn.image = map_photo

news_btn = ctk.CTkButton(card_frame, text="News", image=news_photo, compound="left", font=("Arial", 22, "bold"), width=550, height=80, fg_color="#ff9999", text_color="white", corner_radius=40, command=show_news)
news_btn.pack(pady=12)
news_btn.image = news_photo

weather_container = ctk.CTkFrame(root, fg_color="#f2ffff", width=900, height=700)
weather_container.pack_propagate(False)

weather_scroll_frame = ctk.CTkScrollableFrame(weather_container, width=1080,height=630, fg_color="#f2ffff")
weather_scroll_frame.pack(fill="both", expand=True)

weather_back = ctk.CTkButton(weather_scroll_frame, text="🔙 Back", font=("Arial", 12), command=show_main)

map_container = ctk.CTkFrame(root, fg_color="#f2fff5", width=600, height=600)
map_container.pack_propagate(False)
map_scroll_frame = ctk.CTkScrollableFrame(map_container, width=580, height=580, fg_color="#f2fff5")
map_scroll_frame.pack(fill="both", expand=True)
map_back = ctk.CTkButton(map_scroll_frame, text="🔙 Back", font=("Arial", 12), command=show_main)

news_container = ctk.CTkFrame(root, fg_color="#fff5f5", width=600, height=600)
news_container.pack_propagate(False)
news_scroll_frame = ctk.CTkScrollableFrame(news_container, width=580, height=580, fg_color="#fff5f5")
news_scroll_frame.pack(fill="both", expand=True)
news_back = ctk.CTkButton(news_scroll_frame, text="🔙 Back", font=("Arial", 12), command=show_main)

root.mainloop()


c:\Users\Lenovo\anaconda3\Lib\site-packages\customtkinter\windows\widgets\core_widget_classes\ctk_base_class.py:179: UserWarning: CTkButton Warning: Given image is not CTkImage but <class 'PIL.ImageTk.PhotoImage'>. Image can not be scaled on HighDPI displays, use CTkImage instead.

  warnings.warn(f"{type(self).__name__} Warning: Given image is not CTkImage but {type(image)}. Image can not be scaled on HighDPI displays, use CTkImage instead.\n")
